# A tutorial on understanding fastai apis
> Fastai v2 has impressive APIs and functionality which helps us in writing better, concise and error free code. Lets deep dive into some of these core things here. 

- toc: true 
- badges: true
- comments: true
- hide: true
- search_exclude: false
- categories: [fastai, python, deep-learning]
- image: images/favicon.ico

## Useful decorators

### Delegates 
- usually when we write a function with **kwargs**, it is difficult to get tab completion, so fastcore has implemented [delegates](https://www.fast.ai/2019/08/06/delegation/) function which does this for you. 
- It is extensively used in fastai2.

In [1]:
import inspect
from fastcore.foundation import delegates


def create_web_page(title, category="General", date=None, author="Jeremy"):
    ...

@delegates(create_web_page)
def create_product_page(title, price, cost, **kwargs):
    ...
    
def create_product_page2(title, price, cost, **kwargs):
    ...
    

print(inspect.signature(create_product_page))
print(inspect.signature(create_product_page2))

(title, price, cost, category='General', date=None, author='Jeremy')
(title, price, cost, **kwargs)


### @patch,  @patch_to and @patch_property
- @patch and @patch_to tries to add methods to the already defined classes. We have used patch to add **pad_resize** to Image module. 
- @patch_property adds attributes to already defined classes. suppose you have PIL.Image which doesn't have shape property. You want add this method to the PIL.Image, how can we do this?

In [2]:
from PIL import Image
from fastcore.foundation import patch_property, patch

@patch_property
def shape(x: Image.Image): return (x.size[0], x.size[1])

@patch
def pad_resize(self: Image, size: tuple):
    "Does padding and resize based on h/w"
    return self

In [3]:
f = Image.open("../images/favicon.ico")
print(f.shape)
Image.pad_resize(f, size=(20, 20)) #

(25, 35)


### @typedispatch
- whenever we want to make a function work depending on the input type we can use typedispatch. say for example we want to add two numbers only if their types are both the same. 
- It can be heavily used in augmentations in DL vision, where we implement different functions for say Resize of mask and Resize of image. Using typedispatch, makes us call only one function called resize, but appropriate resize happens after checking the type.

In [4]:
from fastcore.dispatch import typedispatch
@typedispatch
def f(x: int, y: int): return x+y

@typedispatch
def f(x, y): return "It doesn't work like this"

In [5]:
print(f(1, 2))
print(f(1, 2.))

3
It doesn't work like this


In [6]:
@typedispatch
def f2(x: int): return x

In [7]:
f2(9.0)

9.0

## @Transform
Transform uses encodes, decodes and setup to make transformations to input objects.
- encodes: encodes the input, say resize the image from (H, W) to (h, w)
- decodes: decodes the output of encodes to inputs of encodes. resize back from (h,w) to (H, W)
- setups: can be used to setup attributes when the function is defined

Note:
- if a tuple with different types are sent, only those types matching with encodes are processed. 
- input types are maintained for output also. if we don't want this, we can use -> None

In [11]:
from fastcore.transform import Transform

In [24]:
class A(Transform):
    def encodes(self, x:(int,float)): return x/2
    def encodes(self, x:(str,list)): return str(x)+'1'
    def decodes(self, x: (int, float)): return x*2

In [28]:
f = A()
print(f(3))
print(f("a"))
print(f([1, 2]))
print(f.decode(1.5))
print(f.decode("a1"))
print(f((2, "a")))
print(f.decode((2, "a")))

1.5
a1
[1, 2]1
3.0
a1
(1.0, 'a1')
(4, 'a')


## Useful functions

### Chunks
converting a list of lists to list for calculating length or passing it to dataloaders is expensive. fastai chunks makes this simple.

In [8]:
from fastai2.torch_core import Chunks

In [9]:
a = [[1, 2], [3, 4]]
b = Chunks(a)

In [10]:
print(b.lens)
print(b.chunks)
print(b.cumlens)
print(b.totlen)

(#2) [2,2]
[[1, 2], [3, 4]]
[0 2 4]
4


## Important points 
- use fastai2.torch_core.Module instead of torch.nn.Module so that we need not use super

- This is under active development. 

<script src="https://utteranc.es/client.js"
        repo="carbon2silicon"
        issue-term="pathname"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>